# **1. Retrieving data from multiple Webpages**

In [3]:
# connect to the website
import requests
website_url = 'http://web.archive.org/web/20240401160228/https://en.wikipedia.org/wiki/DAX'
response = requests.get(website_url)  # get content of website
response.raise_for_status()  # give error if request failed

# get the table
from bs4 import BeautifulSoup  # import parser
soup = BeautifulSoup(response.text) # parse website
table = soup.find(id='constituents')  # select the table using the id

In [4]:
for link in table.find_all('a'):
    print(link.attrs)

{'href': '/web/20240401160228/https://en.wikipedia.org/wiki/Prime_Standard', 'title': 'Prime Standard'}
{'href': '#endnote_1'}
{'href': '/web/20240401160228/https://en.wikipedia.org/wiki/File:Adidas-group-logo-fr.svg', 'class': ['mw-file-description']}
{'href': '/web/20240401160228/https://en.wikipedia.org/wiki/Adidas', 'title': 'Adidas'}
{'rel': ['nofollow'], 'class': ['external', 'text'], 'href': 'http://web.archive.org/web/20240401160228/http://www.boerse-frankfurt.de/en/equities/search/result?name_isin_wkn=ADS.DE'}
{'href': '/web/20240401160228/https://en.wikipedia.org/wiki/File:Airbus_Logo_2017.svg', 'class': ['mw-file-description']}
{'href': '/web/20240401160228/https://en.wikipedia.org/wiki/Airbus', 'title': 'Airbus'}
{'rel': ['nofollow'], 'class': ['external', 'text'], 'href': 'http://web.archive.org/web/20240401160228/http://www.boerse-frankfurt.de/en/equities/search/result?name_isin_wkn=AIR.DE'}
{'href': '/web/20240401160228/https://en.wikipedia.org/wiki/File:Allianz.svg', 'c

In [5]:
links_wiki = [link.attrs['href'] for link in table.find_all('a') if not 'class' in link.attrs]
links_wiki[:5]

['/web/20240401160228/https://en.wikipedia.org/wiki/Prime_Standard',
 '#endnote_1',
 '/web/20240401160228/https://en.wikipedia.org/wiki/Adidas',
 '/web/20240401160228/https://en.wikipedia.org/wiki/Airbus',
 '/web/20240401160228/https://en.wikipedia.org/wiki/Allianz']

In [6]:
links_wiki.remove('/web/20240401160228/https://en.wikipedia.org/wiki/Prime_Standard')
links_wiki.remove('#endnote_1')
links_wiki[:5]

['/web/20240401160228/https://en.wikipedia.org/wiki/Adidas',
 '/web/20240401160228/https://en.wikipedia.org/wiki/Airbus',
 '/web/20240401160228/https://en.wikipedia.org/wiki/Allianz',
 '/web/20240401160228/https://en.wikipedia.org/wiki/BASF',
 '/web/20240401160228/https://en.wikipedia.org/wiki/Bayer']

In [7]:
base_url = 'https://web.archive.org'
link = links_wiki[0]

response = requests.get(base_url+link)
response.raise_for_status()

In [ ]:
import pandas as pd
df_table = pd.read_html(response.text, attrs={'class':'infobox vcard'})[0]
df_table

In [9]:
df_table = df_table.loc[~df_table[0].isna(),:]
df_table = df_table.iloc[:, :2]
df_table

,0,1
1,"Factory outlet in Herzogenaurach, Germany","Factory outlet in Herzogenaurach, Germany"
2,Formerly,Gebrüder Dassler Schuhfabrik (1924–1949)
3,Company type,Public (AG)
4,Traded as,FWB: ADS DAX component
5,Industry,"Textile, footwear"
6,Founded,"July 1924; 99 years ago in Herzogenaurach, Ger..."
7,Founder,Adolf Dassler
8,Headquarters,"Herzogenaurach, Bavaria, Germany"
9,Area served,Worldwide
10,Key people,Thomas Rabe (chairman)[2] Bjørn Gulden (CEO)


In [10]:
# extract company name from link
company_name = link.split('/')[-1]
company_name

'Adidas'

In [11]:
df_table.columns=['key',company_name]
df_table = df_table.set_index('key')
df_table

,Adidas
key,
"Factory outlet in Herzogenaurach, Germany","Factory outlet in Herzogenaurach, Germany"
Formerly,Gebrüder Dassler Schuhfabrik (1924–1949)
Company type,Public (AG)
Traded as,FWB: ADS DAX component
Industry,"Textile, footwear"
Founded,"July 1924; 99 years ago in Herzogenaurach, Ger..."
Founder,Adolf Dassler
Headquarters,"Herzogenaurach, Bavaria, Germany"
Area served,Worldwide


In [12]:
import time 
time.sleep(5)
print('5 seconds passed')

5 seconds passed


In [13]:
response.elapsed.total_seconds()

5.140578

In [14]:
def load_link(link, base_url='https://web.archive.org'):
    """Extracts information in table with class 'infobox vcard' from an archived Wikipedia link. 
    Returns a single column DataFrame with basic company information.
  
    Args:
        link (str): Subpage of archived Wikipedia.
        base_url (str): Wikipedia main page defaults to 'https://web.archive.org'.
 
    Returns:
        DataFrame: Shape (x,1) with column name [company_name] and keys as index.
    """
    #connect to page
    response = requests.get(base_url+link)
    
    #raise error if no connection
    response.raise_for_status()
    
    #extact table
    df_table = pd.read_html(response.text,attrs={'class':'infobox vcard'})[0]
    
    #clean table
    df_table = df_table.loc[~df_table[0].isna()]
    df_table = df_table.iloc[:, :2]
    company_name = link.split('/')[-1]
    df_table.columns=['key',company_name]
    df_table = df_table.set_index('key')
    
    #add delay
    time.sleep(response.elapsed.total_seconds()*5)
    
    #remove duplicated values
    df_table = df_table.loc[~df_table.index.duplicated(keep='last')]
    
    return df_table

In [ ]:
(load_link(links_wiki[0]) != df_table).sum()

In [ ]:
company_dfs = []
for link in links_wiki:
    print(link)
    df = load_link(link)
    company_dfs.append(df)

In [17]:
df_company = pd.concat(company_dfs, axis=1)
df_company.head()

,Adidas,Airbus,Allianz,BASF,Bayer,Beiersdorf,BMW,Brenntag,Commerzbank,Continental_AG,...,Rheinmetall,RWE,SAP,Sartorius_AG,Siemens,Siemens_Healthineers,Symrise,Volkswagen_Group,Vonovia,Zalando
key,,,,,,,,,,,,,,,,,,,,,
"Factory outlet in Herzogenaurach, Germany","Factory outlet in Herzogenaurach, Germany",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Formerly,Gebrüder Dassler Schuhfabrik (1924–1949),Parent company: European Aeronautic Defence an...,NaN,NaN,NaN,NaN,Rapp Moterenwerke AG,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deutsche Annington,NaN
Company type,Public (AG),Public (Societas Europaea),Public (SE),Public (Societas Europaea),Public,Public (AG),Public (Aktiengesellschaft),Public (Societas Europaea),NaN,Aktiengesellschaft,...,Public (Aktiengesellschaft),Aktiengesellschaft,Public (Societas Europaea),Aktiengesellschaft,Public (Aktiengesellschaft),Public (Aktiengesellschaft),Aktiengesellschaft,Public (AG),NaN,Societas Europaea
Traded as,FWB: ADS DAX component,BMAD: AIR Euronext Paris: AIR FWB: AIR CAC 40 ...,FWB: ALVDAX component,FWB: BASDAX component,FWB: BAYNDAX componentEuro Stoxx 50 component,FWB: BEI DAX Component,FWB: BMW DAX component,FWB: BNR FWB: BNRA (ADR) OTC Pink Current: BNT...,FWB: CBK DAX Component LSE: CZB,FWB: CON DAX Component,...,FWB: RHMFWB: RHMB (ADR)OTC Pink Current: RNMBF...,FWB: RWE FWB: RWEA (ADR) DAX component (RWE),FWB: SAPFWB: SAPA (ADR)OTC Pink Current: SAPGF...,FWB: SRTFWB: SRT3FWB: SRT0 (ADR)FWB: SRT4 (ADR...,FWB: SIEDAX component,FWB: SHL DAX component,FWB: SY1 FWB: SY1U (ADR) OTC Pink Current: SYI...,"FWB: VOW, VOW3 DAX component (VOW3)",FWB: VNA DAX Component,FWB: ZAL DAX Component
Industry,"Textile, footwear","Aerospace, Defence",Financial services,Chemicals,Pharmaceuticals Chemicals Biotechnology Health...,Consumer goods,Automotive,Distribution,Financial services,Automotive Manufacturing,...,Defence,Electricity generation,Enterprise softwareBusiness softwareCloud comp...,Pharmaceutical and Laboratory Equipment,Conglomerate,Healthcare,Chemicals,Manufacturing,Real estate,E-commerce


In [18]:
df_company = df_company.T
df_company.head()

key,"Factory outlet in Herzogenaurach, Germany",Formerly,Company type,Traded as,Industry,Founded,Founder,Headquarters,Area served,Key people,...,Native name,Headquarters in Stuttgart,"Porsche headquarters in Stuttgart, Germany",Footnotes / references [1][2],Logo since 2011,"Headquarters in Wolfsburg, Germany",Headquarters of Vonovia,Areas served,Footnotes / references Annual Report 2020,Headquarters in Berlin
Adidas,"Factory outlet in Herzogenaurach, Germany",Gebrüder Dassler Schuhfabrik (1924–1949),Public (AG),FWB: ADS DAX component,"Textile, footwear","July 1924; 99 years ago in Herzogenaurach, Ger...",Adolf Dassler,"Herzogenaurach, Bavaria, Germany",Worldwide,Thomas Rabe (chairman)[2] Bjørn Gulden (CEO),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Airbus,NaN,Parent company: European Aeronautic Defence an...,Public (Societas Europaea),BMAD: AIR Euronext Paris: AIR FWB: AIR CAC 40 ...,"Aerospace, Defence",18 December 1970; 53 years ago,NaN,"Blagnac (n. Toulouse), France (operational) Le...",Worldwide,Guillaume Faury (CEO) René Obermann (Chairman),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Allianz,NaN,NaN,Public (SE),FWB: ALVDAX component,Financial services,05 February 1890; 134 years ago,NaN,"Munich, Germany",Worldwide,Oliver Bäte (CEO)Giulio Terzariol (CFO)Ishara ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BASF,NaN,NaN,Public (Societas Europaea),FWB: BASDAX component,Chemicals,6 April 1865; 158 years ago (as Badische Anili...,Friedrich Engelhorn,"Ludwigshafen, Germany",NaN,Jürgen Hambrecht (chairman of the supervisory ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bayer,NaN,NaN,Public,FWB: BAYNDAX componentEuro Stoxx 50 component,Pharmaceuticals Chemicals Biotechnology Health...,1 August 1863; 160 years ago[1],Friedrich Bayer,"Leverkusen, North Rhine-Westphalia, Germany",Worldwide,Bill Anderson (CEO) Norbert Winkeljohann (Chai...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_company.isna().sum()

key
Factory outlet in Herzogenaurach, Germany    36
Formerly                                     31
Company type                                  3
Traded as                                     1
Industry                                      1
                                             ..
Headquarters in Wolfsburg, Germany           36
Headquarters of Vonovia                      36
Areas served                                 36
Footnotes / references Annual Report 2020    36
Headquarters in Berlin                       36
Length: 77, dtype: int64

In [20]:
cols = ['Revenue', 'Operating income', 'Net income', 'Total assets', 'Total equity']
df_company.loc[:,cols]

key,Revenue,Operating income,Net income,Total assets,Total equity
Adidas,€21.915 billion (2018)[3],€2.368 billion (2018)[3],€1.702 billion (2018)[3],€15.612 billion (2018)[3],€6.364 billion (2018)[3]
Airbus,€65.45 billion (2023),€4.60 billion (2023),€3.79 billion (2023),€118.87 billion (2023),€17.73 billion (2023)
Allianz,€152.7 billion (2022),€14.16 billion (2022),€7.18 billion (2022),€1.02 trillion (2022),€51.0 billion (2022)
BASF,€87.3 billion (2022)[1],€6.55 billion (2022)[1],€−627 million (2022)[1],€84.5 billion (2022)[1],€40.9 billion (2022)[1]
Bayer,€50.74 billion (2023)[2],€7.01 billion (2022)[2],€4.15 billion (2022)[2],€124.9 billion (2022)[2],€38.93 billion (2022)[2]
Beiersdorf,€7.653 billion (2019)[1],€1.095 billion (2019)[1],€736 million (2019)[1],€9.63 billion (2019)[2],NaN
BMW,€142.610 billion (2022)[1],€23.509 billion (2022)[1],€18.582 billion (2022)[1],€246.926 billion (2022)[1],€91.288 billion (2022)[1]
Brenntag,19.4 billion EUR (2022)[1],NaN,NaN,NaN,NaN
Commerzbank,€8.57 billion (2018),€1.245 billion (2018),€865 million (2018),€462 billion (2018),€29 billion (2018)
Continental_AG,"€41,030 million (2022)",€754.8 million (2022),€66.6 million (2022),"€37,926.7 million (2022)","€13,735.0 million (2022)"


In [21]:
# df_company.to_pickle('company_data.p')

In [22]:
df_company = df_company.loc[:, ['Operating income', 'Net income', 'Revenue', 'Total assets', 'Total equity']]
df_company.head()

key,Operating income,Net income,Revenue,Total assets,Total equity
Adidas,€2.368 billion (2018)[3],€1.702 billion (2018)[3],€21.915 billion (2018)[3],€15.612 billion (2018)[3],€6.364 billion (2018)[3]
Airbus,€4.60 billion (2023),€3.79 billion (2023),€65.45 billion (2023),€118.87 billion (2023),€17.73 billion (2023)
Allianz,€14.16 billion (2022),€7.18 billion (2022),€152.7 billion (2022),€1.02 trillion (2022),€51.0 billion (2022)
BASF,€6.55 billion (2022)[1],€−627 million (2022)[1],€87.3 billion (2022)[1],€84.5 billion (2022)[1],€40.9 billion (2022)[1]
Bayer,€7.01 billion (2022)[2],€4.15 billion (2022)[2],€50.74 billion (2023)[2],€124.9 billion (2022)[2],€38.93 billion (2022)[2]


In [23]:
df_company.loc[:, 'Total assets year'] = df_company.loc[:, 'Total assets'].str.extract(r'(\d{4})', expand=False)
df_company.loc[:, 'Total assets year']

Adidas                   2018
Airbus                   2023
Allianz                  2022
BASF                     2022
Bayer                    2022
Beiersdorf               2019
BMW                      2022
Brenntag                  NaN
Commerzbank              2018
Continental_AG           2022
Covestro                 2022
Daimler_Truck             NaN
Deutsche_Bank            2023
Deutsche_B%C3%B6rse      2022
Deutsche_Post             NaN
Deutsche_Telekom         2022
E.ON                     2022
Hannover_Re               NaN
Henkel                   2023
Infineon_Technologies    2023
Mercedes-Benz_Group      2021
Merck_Group              2023
MTU_Aero_Engines         2019
Munich_Re                2022
Porsche                  2022
Porsche_SE               2021
Qiagen                   2023
Rheinmetall              2022
RWE                      2019
SAP                      2023
Sartorius_AG              NaN
Siemens                  2023
Siemens_Healthineers     2022
Symrise   

In [24]:
df_company.loc[:, 'Total assets value'] = df_company.loc[:, 'Total assets'].str.extract(r'([\d.,]+\s?[a-zA-Z]+)', expand=False)
df_company.loc[:, 'Total assets value']

Adidas                     15.612 billion
Airbus                     118.87 billion
Allianz                     1.02 trillion
BASF                         84.5 billion
Bayer                       124.9 billion
Beiersdorf                   9.63 billion
BMW                       246.926 billion
Brenntag                              NaN
Commerzbank                   462 billion
Continental_AG           37,926.7 million
Covestro                     14.6 billion
Daimler_Truck                         NaN
Deutsche_Bank               1.31 trillion
Deutsche_B%C3%B6rse         269.1 billion
Deutsche_Post                         NaN
Deutsche_Telekom            298.6 billion
E.ON                      134.009 billion
Hannover_Re                           NaN
Henkel                     17.965 billion
Infineon_Technologies      28.439 billion
Mercedes-Benz_Group         258.8 billion
Merck_Group                 48.49 billion
MTU_Aero_Engines            7.765 billion
Munich_Re                   298.5 

In [ ]:
df_company.loc[:,'Total assets value'] = df_company.loc[:,'Total assets value'].str.replace('\strillion','e12', regex=True)  # replace trillion with e12
df_company.loc[:,'Total assets value'] = df_company.loc[:,'Total assets value'].str.replace('\sbillion','e9', regex=True)  # replace billion with e9
df_company.loc[:,'Total assets value'] = df_company.loc[:,'Total assets value'].str.replace('\smillion','e6', regex=True)  # replace million with e6
df_company.loc[:,'Total assets value'] = df_company.loc[:,'Total assets value'].str.replace(',','', regex=True)  # deleting , due to english thousands separator notation
df_company.loc[:,'Total assets value'] = df_company.loc[:,'Total assets value'].astype(float)  # convert to float
df_company.loc[:,'Total assets value']

In [26]:
mask_dollar = df_company.loc[:, 'Total assets'].str.strip().str.startswith('US$', na=False)
mask_dollar

Adidas                   False
Airbus                   False
Allianz                  False
BASF                     False
Bayer                    False
Beiersdorf               False
BMW                      False
Brenntag                 False
Commerzbank              False
Continental_AG           False
Covestro                 False
Daimler_Truck            False
Deutsche_Bank            False
Deutsche_B%C3%B6rse      False
Deutsche_Post            False
Deutsche_Telekom         False
E.ON                     False
Hannover_Re              False
Henkel                   False
Infineon_Technologies    False
Mercedes-Benz_Group      False
Merck_Group              False
MTU_Aero_Engines         False
Munich_Re                False
Porsche                  False
Porsche_SE               False
Qiagen                    True
Rheinmetall              False
RWE                      False
SAP                      False
Sartorius_AG             False
Siemens                  False
Siemens_

In [27]:
df_company.loc[mask_dollar, 'Total assets value'] = df_company.loc[mask_dollar, 'Total assets value'] * 0.95
df_company.loc[mask_dollar, 'Total assets value']

Qiagen    5814000000.0
Name: Total assets value, dtype: object

In [ ]:
for col in ['Operating income', 'Net income', 'Revenue', 'Total equity']:  # repeat for all columns with financial data
    print(col)
    df_company.loc[:, col+' year'] = df_company.loc[:, col].str.extract(r'(\d{4})', expand=False)  # extract the year
    df_company.loc[:, col+' value'] = df_company.loc[:, col].str.extract(r'([\d.,]+\s?[a-zA-Z]+)', expand =False)  # extract the value
    
    df_company.loc[:, col+' value'] = df_company.loc[:, col+' value'].str.replace('\strillion','e12', regex=True)  # replace trillion with e12
    df_company.loc[:, col+' value'] = df_company.loc[:, col+' value'].str.replace('\sbillion','e9', regex=True)  # replace billion with e9
    df_company.loc[:, col+' value'] = df_company.loc[:, col+' value'].str.replace('\smillion','e6', regex=True)  # replace million with e6
    df_company.loc[:, col+' value'] = df_company.loc[:, col+' value'].str.replace('\sMio', 'e6', regex=True) # replace Mio with e6
    df_company.loc[:, col+' value'] = df_company.loc[:, col+' value'].str.replace('M','e6', regex=True)  # replace M with e6
    df_company.loc[:, col+' value'] = df_company.loc[:, col+' value'].str.replace('B','e9', regex=True)  # replace B with e9
    df_company.loc[:, col+' value'] = df_company.loc[:, col+' value'].str.replace('\seuro','', regex=True)  # replace , with .
    df_company.loc[:, col+' value'] = df_company.loc[:, col+' value'].str.replace(',','', regex=True)  # replace , with .
    df_company.loc[:, col+' value'] = df_company.loc[:, col+' value'].astype(float)  # convert to float
    
    mask_dollar = df_company.loc[:, col].str.strip().str.startswith('US$', na=False)  # create mask to select the $-values
    df_company.loc[mask_dollar, col+' value'] = df_company.loc[mask_dollar, col+' value'] * 0.95  # calculate the €-values fromt the $-values

In [29]:
df_company = df_company.drop(['Operating income', 'Net income', 'Revenue', 'Total assets', 'Total equity'], axis=1)
df_company.head()

key,Total assets year,Total assets value,Operating income year,Operating income value,Net income year,Net income value,Revenue year,Revenue value,Total equity year,Total equity value
Adidas,2018,15612000000.0,2018,2368000000.0,2018,1702000000.0,2018,21915000000.0,2018,6364000000.0
Airbus,2023,118870000000.0,2023,4600000000.0,2023,3790000000.0,2023,65450000000.0,2023,17730000000.0
Allianz,2022,1020000000000.0,2022,14160000000.0,2022,7180000000.0,2022,152700000000.0,2022,51000000000.0
BASF,2022,84500000000.0,2022,6550000000.0,2022,627000000.0,2022,87300000000.0,2022,40900000000.0
Bayer,2022,124900000000.0,2022,7010000000.0,2022,4150000000.0,2023,50740000000.0,2022,38930000000.0


In [ ]:
# df_company.to_pickle('dax_financial_data.p')